In [1]:
import json
import base64
import urllib3
import requests
import warnings
import pandas as pd
from urllib3.exceptions import InsecureRequestWarning

# Suppress only the InsecureRequestWarning caused by unverified HTTPS requests
warnings.simplefilter('ignore', InsecureRequestWarning)

# Disable all urllib3 warnings (optional, you may not want to do this in a production environment)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# movies
movies_service_token = 'eyJ0eXBlIjoiaHBhZ2UiLCJuYXYiOmZhbHNlLCJzZWMiOiJjZW50ZXIiLCJwcmlkIjoiYzliMmUxODEtZmNkNi00YzFkLTlmYjktMWYwZmEwOTc5YjVlIiwib2Zmc2V0IjowLCJucHNpIjozMjAsIm9yZXEiOiIwMWVmNjBmNS1kNmFiLTQ2NDktYmNiYy0wZDU3NDA1NjQxZGM6MTcwNTQ1MDgyMTAwMCIsImFwTWF4IjoyNTEwMCwic3RyaWQiOiIxOjEyNkdXSlJKVTU0QldRIyNPWlNYRTVESk1OUVdZVERKT04yQSIsInN0S2V5Ijoie1wicXVlcnlcIjpcImZpZWxkLWVudGl0eV90eXBlPU1vdmllJnB2X29mZmVycz1JTiUzQSUzQSUzQSUzQXZvZCUzQS0xNzA2MDU1NjAwJTNBMTcwNTQ1MDgwMC0mc29ydD1mZWF0dXJlZC1yYW5rJnNlYXJjaC1hbGlhcz1pbnN0YW50LXZpZGVvXCIsXCJxdWVyeVR5cGVcIjpcIlNUUlVDVFVSRURcIixcInJhbmtpbmdTdHJhdGVneVwiOlwiREVGQVVMVFwiLFwibmV4dFN0YXJ0SW5kZXhcIjozMjB9Iiwib3JlcWsiOiJqc0RqMFFsMHpyV0JZRFR5SHE4UG5EcW1HN2ZySmkzNXJ4ekV6REV6aXJJPSIsIm9yZXFrdiI6MX0='

# tv shows
tv_shows_service_token = ''


In [2]:
def generate_service_token():
    url = "https://www.primevideo.com/search/ref=atv_tv_hom_c_laptvn_7_smr?queryToken=eyJ0eXBlIjoicXVlcnkiLCJuYXYiOnRydWUsInBpIjoiZGVmYXVsdCIsInNlYyI6ImNlbnRlciIsInN0eXBlIjoic2VhcmNoIiwicXJ5IjoicF9uX2VudGl0eV90eXBlPVRWIFNob3cmaW5kZXg9ZXUtYW1hem9uLXZpZGVvLW90aGVyJmFkdWx0LXByb2R1Y3Q9MCZicT0oYW5kIHNvcnQ6J2ZlYXR1cmVkLXJhbmsnIChub3QgYXZfcHJpbWFyeV9nZW5yZTonY2hpbGRyZW4nKSkmYXZfYnlfdHlwZV9pbl90ZXJyaXRvcnk9SU46c3ZvZDpwcmltZSZzZWFyY2gtYWxpYXM9aW5zdGFudC12aWRlbyZxcy1hdl9yZXF1ZXN0X3R5cGU9NCZxcy1pcy1wcmltZS1jdXN0b21lcj0yIiwicnQiOiJsQVB0Vm5zbXIiLCJ0eHQiOiJUb3AgVFYiLCJvZmZzZXQiOjAsIm5wc2kiOjAsIm9yZXEiOiIzNzQ0NTc2NS1mMDY4LTQ3MzctYjlkZi01NzNlOTI0NmRmOWQ6MTYxMDY0MTI3NDAwMCIsInN0cmlkIjoiMToxSUFZRzBCVVQxRVZPIyNNWlFXR1pMVU1WU0VHWUxTTjUyWEdaTE0iLCJvcmVxayI6ImpzRGowUWwwenJXQllEVHlIcThQbkRxbUc3ZnJKaTM1cnh6RXpERXppckk9Iiwib3JlcWt2IjoxfQ%3D%3D&pageId=default&queryPageType=browse&ie=UTF8"
    resp = requests.get(url, verify=False)
    return resp.url.split("serviceToken=", 1)[-1].split("%3D", 1)[0]
    

def get_service_token(startindex, content_type):
    # Replace the encoded token with your actual token
    
    if content_type == "Movies":
        encoded_token = movies_service_token
    else:
        encoded_token = tv_shows_service_token

    # encoded_token = generate_service_token()

    # Add padding to the encoded string if needed
    padding = len(encoded_token) % 4
    if padding:
        encoded_token += '=' * (4 - padding)

    # Decode the Base64 string
    decoded_bytes = base64.urlsafe_b64decode(encoded_token)
    decoded_string = decoded_bytes.decode('utf-8')

    # Parse the JSON data
    decoded_data = json.loads(decoded_string)

    # updating decoded data
    decoded_data["npsi"] = startindex


    return base64.b64encode(json.dumps(decoded_data).encode()).decode()


def get_page_load_metadata(service_token, startindex=0):

    headers = {
        'Accept': '*/*',
        'Accept-Language': 'en-GB,en;q=0.5',
        'Connection': 'keep-alive',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-GPC': '1',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Brave";v="120"',
        'sec-ch-ua-mobile': '?0',
        'x-requested-with': 'XMLHttpRequest',
    }

    params = {
        'pageType': 'search',
        'pageId': 'flex',
        'collectionType': 'Container',
        'paginationTargetId': 'V2=4AEA6u69gYPeuYe-toZwYWdlSWSIcGFnZVR5cGWMY29sbGVjdGlvbklkiHdpZGdldElkjo5zd2lmdElkVmVyc2lvbt6-iodkZWZhdWx0i4Zicm93c2WMjqRiMDRhMDY0Ni05ODA4LTQyZWYtYjU5My1iMzQ2YzAyNTNkNWWND46CVjI=',
        'serviceToken': service_token,
        'startIndex': str(startindex),
        'variant': 'desktopOSX',
        'actionScheme': 'default',
        'payloadScheme': 'default',
        'decorationScheme': 'web-search-decoration-tournaments-v2',
        'featureScheme': 'web-search-v2',
        'dynamicFeatures': [
            'integration',
            'CLIENT_DECORATION_ENABLE_DAAPI',
            'ENABLE_DRAPER_CONTENT',
            'HorizontalPagination',
            'CleanSlate',
            'EpgContainerPagination',
            'ENABLE_GPCI',
            'SupportsImageTextLinkTextInStandardHero',
        ],
        'widgetScheme': 'web-explorecs-v10',
        'journeyIngressContext': '',
    }

    response = requests.get('https://www.primevideo.com/api/paginateCollection', params=params, headers=headers, verify=False)
    return response.json()



# id,title,type,release_year,runtime,backdrop_path,made_by_netflix,maturity_rating,maturity_warning,maturity_rating_reason,maturity_rating_board,overview,genres,cast,content_is,subtitles,audio
def process_titles(title_items):
    title_list = []
    for title_info in title_items:
        try:
            title_list.append({
                "title_id": title_info["titleID"],
                "name": title_info["title"],
                "description": title_info["synopsis"],
                "type": title_info["entityType"],
                "entitlements": title_info["entitlements"],
                "release_year": title_info.get("releaseYear"),
                "primary_genre": title_info.get("categorizedGenres", {}).get("primaryGenre"),
                "secondary_genre": title_info.get("categorizedGenres", {}).get("secondaryGenres"),
                "maturity_rating": title_info["contentMaturityRating"]["rating"],
                "backdrop_path": title_info["image"]["url"],
                "url": f"https://www.primevideo.com{title_info['link']['url']}",
            })
        except Exception as e:
            print(title_info)
            raise e
    return title_list
    

def main():
    # List to store information about titles
    title_info_list = []

    # Specify the content type ("Movies" or "TV Shows")
    content_type = "Movies"
    valid_content_types = ("Movies", "TV Shows")
    
    # Check if the specified content type is valid
    if content_type not in valid_content_types:
        raise Exception(f"Invalid content type: '{content_type}'. Input correct content type. Possible options are: {valid_content_types}")

    # Print information about the content type being processed
    print(f"Processing title information for content type: {content_type}")

    # Initialize the start index for pagination
    startindex = 0

    # Main loop for fetching title information
    while True:
        # Get the service token for the specified content type and start index
        service_token = get_service_token(startindex, content_type)

        # Fetch page load metadata using the service token and start index
        response = get_page_load_metadata(service_token, startindex=startindex)
        
        # Process title information if there are items in the response
        if response["items"]:
            title_items = response["items"]
            title_info_list.extend(process_titles(title_items))

        # Update the start index for the next iteration
        startindex = response.get("pagination", {}).get("startIndex", None)

        # Break the loop if there are no more items to process
        if not response["hasMoreItems"]:
            break

        # Print progress information
        print(f"processed titles -> {len(title_info_list)}")

    # Convert the list of title information to a Pandas DataFrame
    df = pd.DataFrame.from_records(title_info_list)
    # print(df)

    # Print summary information if the DataFrame is not empty
    print("*******************************************")
    if not df.empty:
        # Count the occurrences of each title type
        type_counts = df['type'].value_counts()
        print(f"total no of titles: {len(df)}")
        print(f"titles type name and count: {type_counts}")

        # Save the DataFrame to a CSV file
        csv_filename = f"prime_video_{content_type.replace(' ', '_').lower()}.csv"
        df.to_csv(csv_filename, index=False)
        print(f"CSV file: '{csv_filename}' saved successfully!")
        return df
    else:
        # Print a message if there are no titles to save
        print("nothing to save...")
        return


if __name__ == "__main__":
    # Call the main function when the script is executed
    main()


Processing title information for content type: Movies
processed titles -> 40
processed titles -> 80
processed titles -> 120
processed titles -> 160
processed titles -> 200
processed titles -> 240
processed titles -> 280
processed titles -> 320
processed titles -> 360
processed titles -> 400
processed titles -> 440
processed titles -> 480
processed titles -> 520
processed titles -> 560
processed titles -> 600
processed titles -> 640
processed titles -> 680
processed titles -> 720
processed titles -> 760
processed titles -> 800
processed titles -> 840
processed titles -> 880
processed titles -> 920
processed titles -> 960
processed titles -> 1000
processed titles -> 1040
processed titles -> 1080
processed titles -> 1120
processed titles -> 1160
processed titles -> 1200
processed titles -> 1240
processed titles -> 1280
processed titles -> 1320
processed titles -> 1360
processed titles -> 1400
processed titles -> 1440
processed titles -> 1480
processed titles -> 1520
processed titles -> 15

In [3]:
# s = "eyJ0eXBlIjoiaHBhZ2UiLCJuYXYiOmZhbHNlLCJzZWMiOiJjZW50ZXIiLCJwcmlkIjoiYzliMmUxODEtZmNkNi00YzFkLTlmYjktMWYwZmEwOTc5YjVlIiwib2Zmc2V0IjowLCJucHNpIjozMjAsIm9yZXEiOiIwMWVmNjBmNS1kNmFiLTQ2NDktYmNiYy0wZDU3NDA1NjQxZGM6MTcwNTQ1MDgyMTAwMCIsImFwTWF4IjoyNTEwMCwic3RyaWQiOiIxOjEyNkdXSlJKVTU0QldRIyNPWlNYRTVESk1OUVdZVERKT04yQSIsInN0S2V5Ijoie1wicXVlcnlcIjpcImZpZWxkLWVudGl0eV90eXBlPU1vdmllJnB2X29mZmVycz1JTiUzQSUzQSUzQSUzQXZvZCUzQS0xNzA2MDU1NjAwJTNBMTcwNTQ1MDgwMC0mc29ydD1mZWF0dXJlZC1yYW5rJnNlYXJjaC1hbGlhcz1pbnN0YW50LXZpZGVvXCIsXCJxdWVyeVR5cGVcIjpcIlNUUlVDVFVSRURcIixcInJhbmtpbmdTdHJhdGVneVwiOlwiREVGQVVMVFwiLFwibmV4dFN0YXJ0SW5kZXhcIjozMjB9Iiwib3JlcWsiOiJqc0RqMFFsMHpyV0JZRFR5SHE4UG5EcW1HN2ZySmkzNXJ4ekV6REV6aXJJPSIsIm9yZXFrdiI6MX0="
# padding = len(s) % 4
# if padding:
#     s += '=' * (4 - padding)

# json.loads(base64.urlsafe_b64decode(s).decode("utf-8"))

{'type': 'hpage',
 'nav': False,
 'sec': 'center',
 'prid': 'c9b2e181-fcd6-4c1d-9fb9-1f0fa0979b5e',
 'offset': 0,
 'npsi': 320,
 'oreq': '01ef60f5-d6ab-4649-bcbc-0d57405641dc:1705450821000',
 'apMax': 25100,
 'strid': '1:126GWJRJU54BWQ##OZSXE5DJMNQWYTDJON2A',
 'stKey': '{"query":"field-entity_type=Movie&pv_offers=IN%3A%3A%3A%3Avod%3A-1706055600%3A1705450800-&sort=featured-rank&search-alias=instant-video","queryType":"STRUCTURED","rankingStrategy":"DEFAULT","nextStartIndex":320}',
 'oreqk': 'jsDj0Ql0zrWBYDTyHq8PnDqmG7frJi35rxzEzDEzirI=',
 'oreqkv': 1}